In [1]:
import numpy as np
import pandas as pd
import re,math
from sklearn import preprocessing
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
from scipy.optimize import leastsq

data1 = pd.read_csv(r'pearson.csv')

In [2]:
# 细胞系-细胞系
high_data = data1.set_index(data1.loc[:,"Unnamed: 0"]).drop(["Unnamed: 0"], axis=1)

In [4]:
# 药物对-细胞系
data2 = pd.read_csv('label_gai.csv')

In [7]:
columns = list(data2.columns[1:])

In [8]:
# 删去对应行后的字典
dict_high_lines = {}
count = 0
for i in high_data.index:
    y = list(high_data.loc[:,i])
    y.pop(count)
    count += 1  
    dict_high_lines[i] = y

In [9]:
# 分离x y数据构造字典
dict_columns = {}
for i in columns: 
    
    # b
    y = data2.loc[:,i]
    
    # A
    temp = []
    for j in columns:
        if j != i:
            temp.append(j)
    m = data2.loc[:, temp[0]]
    tp = [m]
    for j in temp[1:]:
        tp.append(data2.loc[:, j])
    x = pd.DataFrame(tp).T
    

    tp = []
    for j in columns:
        if i != j:
            tp.append(j)
    df = pd.DataFrame(dict_high_lines[i], index=tp)
    df.sort_values(by=0 , inplace=True, ascending=True)
    max_l = df.index[-5:]
    tp = []
    for j in max_l:
        tp.append(data2.loc[:, j])
    max_5x = pd.DataFrame(tp).T
    
    # 存储
    data = {
        "x": x,
        "y": y,
        "max_relevance": max_l,
        "max_5x": max_5x
    }
    dict_columns[i] = data

In [10]:
# 删去对应行后的字典
dict_high_lines = {}
count = 0
for i in high_data.index:
    y = list(high_data.loc[:,i])
    y.pop(count)
    count += 1
    dict_high_lines[i] = y
    
    tp = []
    for j in columns:
        if i != j:
            tp.append(j)
    df = pd.DataFrame(dict_high_lines[i], index=tp)
    df.sort_values(by=0 , inplace=True, ascending=True)
    
    max_l = df.index[-5:]
    tp = []
    for j in max_l:
        tp.append(high_data.loc[j, :])
    max_5y = list(pd.DataFrame(tp).T.loc[[i]].values)[0]
    
    data = {
        "max_relevance": max_l,
        "y": y,
        "high_5": max_5y
    }
    dict_high_lines[i] = data

In [11]:
end_score = []
name_dict_high_lines = []
for elem in dict_high_lines:
    end_score.append(dict_high_lines[elem]['high_5'].mean())
    name_dict_high_lines.append(elem)
dict_high_lines_temp = {}
for score in end_score:
    if score >= 0.92:
        dict_high_lines_temp[name_dict_high_lines[end_score.index(score)]] = dict_high_lines[name_dict_high_lines[end_score.index(score)]]
dict_high_lines = dict_high_lines_temp.copy()

In [12]:
def dictW(alpha):
    # 运算构造参数W字典 
    dict_W = {}
    for elem in dict_high_lines:
        r = np.array(dict_high_lines[elem]["high_5"])
        W1= pow((1-r),2)
        W = np.exp(-W1/(2*pow(alpha,2)))
        dict_W[elem] = W
    return dict_W

def dictr(dict_W):
    # r = max(|A×W - b|)
    dict_r = {}
    for elem,w in zip(dict_columns, dict_W):
        A = np.array(dict_columns[elem]["max_5x"].values) # x----->max_5x
        b = np.array(dict_columns[elem]["y"].values)
        W = dict_W[w]
        temp1 = np.dot(A,W) - b
        temp2 = np.square(temp1)
        temp3 = np.mean(temp2)
        dict_r[w] = temp3
    return dict_r

def dirlist(dict_r):
    #计算误差平方和    
    dir_list = []
    for elem in dict_r:
        dir_list.append(np.sqrt(dict_r[elem]))
    return dir_list

In [13]:
dev=10000000
lin=np.linspace(0.001,0.1,100)
for alpha in lin:
    
    dict_W = dictW(alpha)
    
    dict_r = dictr(dict_W)
 
    dir_list=dirlist(dict_r)

    temp=np.mean(dir_list)
                      
    #误差判断
    if(temp>=dev):
        continue
    else:
        dev=temp
        al=alpha

alpha=al
dict_W = dictW(alpha)
dict_r = dictr(dict_W)

In [14]:
import pickle

with open('细胞系.pkl', 'wb') as file:
    pickle.dump(dict_W, file)
    pickle.dump(dict_columns, file)
    pickle.dump(dict_high_lines, file)

In [15]:
np.array(list(dict_r.values())).mean()

338.82596997210396